In [29]:
from datetime import datetime

from pyspeedy import Speedy
from pyspeedy.callbacks import ModelCheckpoint, XarrayExporter

start_date = datetime(1980, 1, 1)  # Simulation start date (datetime object).
end_date = datetime(2010, 12, 31)  # Simulation end date.
spinup_date = datetime(1980, 2, 1)  # End of spinup period.

# Create an instance of the speedy model.
model = Speedy(
    start_date=start_date,  # Simulation start date (datetime object).
    end_date=end_date,  # Simulation end date.
)
# At this point, the model state is "empty".
# To initialize the model, we need to define its boundary conditions first.
# This function will set the default boundary conditions derived from the ERA reanalysis.
model.set_bc()

# Before running the model, let's initialize two callback functions that

# Initialize the callback functions that saves the model data into netcdf files
# A "callback" is an object that performs user defined actions at each time step.
output_v=('u_grid', 'v_grid', 't_grid', 'q_grid', 'phi_grid', 'ps_grid',\
          'evap_grid','precls')
my_exporter = XarrayExporter(
    output_dir="./data",  # Output directory where the model output will be stored
    interval=36,  # Every how many time steps we will save the output file. 36 -> once per day.
    verbose=True,  # Prind progress messages
    variables=output_v,  # Which variables to output. If none, save the most commonly used variables.
    spinup_date=spinup_date,  # End of spinup period
)

# Let's initialized another callback. This one keeps a dataframe with selected variables
# with different model times ("checkpoints").
# The dataframe with the data is stored in the "dataframe" attribute of the
# created ModelCheckpoint instance.

model_checkpoints = ModelCheckpoint(
    interval=36,  # Every how many time steps we will save the output file. 36 -> once per day.
    verbose=True,  # Prind progress messages
    variables=output_v,  # Which variables to output. If none, save the most commonly used variables.
    spinup_date=spinup_date,  # End of spinup period
)


varList=['phi0','orog','phis0','alb0','forog','fmask_orig',\
         'stl12','snowd12','soilw12','veg_low','veg_high','sst12','sea_ice_frac12','sst_anom',
         'u_grid','v_grid','q_grid','phi_grid','ps_grid','precls','evap']


varList2=['u_grid','v_grid','q_grid','phi_grid','ps_grid', 'tgrid','sst_anom']
itime=0
u_gridL=[]
v_gridL=[]
q_gridL=[]
phi_gridL=[]
ps_gridL=[]
tgridL=[]
sst_anomL=[]
xL=[]
idayL=[]

In [24]:


print(model.current_date)


import xarray as xr
while itime<120*31*36+10*31*60 and model.current_date<end_date:
    old_month=model.current_date.month
    old_year=model.current_date.year
    model.run1()
    new_month=model.current_date.month
    new_year=model.current_date.year
    dmonth=(new_year-1980)*12+new_month
    if itime%9==0:
        model.spectral2grid()
        u_gridL.append(model['u_grid'].copy())
        v_gridL.append(model['v_grid'].copy())
        q_gridL.append(model['q_grid'].copy())
        phi_gridL.append(model['phi_grid'].copy())
        ps_gridL.append(model['ps_grid'].copy())
        tgridL.append(model['t_grid'].copy())
        sst_anomL.append(model['sst_anom'][:,:,dmonth].copy())
        idayL.append((model.current_date-datetime(old_year,1,1,0,0,0)).days)
    if new_month!=old_month:
        ss_u=u_gridL[-1].shape
        u_gridX=xr.DataArray(u_gridL,coords=[range(len(u_gridL)),range(ss_u[0]),range(ss_u[1]),range(ss_u[2])],dims=['time','lon','lat','lev'])
        ss_v=v_gridL[-1].shape
        v_gridX=xr.DataArray(v_gridL,coords=[range(len(v_gridL)),range(ss_v[0]),range(ss_v[1]),range(ss_v[2])],dims=['time','lon','lat','lev'])
        ss_q=q_gridL[-1].shape
        q_gridX=xr.DataArray(q_gridL,coords=[range(len(q_gridL)),range(ss_q[0]),range(ss_q[1]),range(ss_q[2])],dims=['time','lon','lat','lev'])
        ss_phi=phi_gridL[-1].shape
        phi_gridX=xr.DataArray(phi_gridL,coords=[range(len(phi_gridL)),range(ss_phi[0]),range(ss_phi[1]),range(ss_phi[2])],dims=['time','lon','lat','lev'])
        ss_ps=ps_gridL[-1].shape
        ps_gridX=xr.DataArray(ps_gridL,coords=[range(len(ps_gridL)),range(ss_ps[0]),range(ss_ps[1])],dims=['time','lon','lat'])
        ss_t=tgridL[-1].shape
        tgridX=xr.DataArray(tgridL,coords=[range(len(tgridL)),range(ss_t[0]),range(ss_t[1]),range(ss_t[2])],dims=['time','lon','lat','lev'])
        ss_sst=sst_anomL[-1].shape
        sst_anomX=xr.DataArray(sst_anomL,coords=[range(len(sst_anomL)),range(ss_sst[0]),range(ss_sst[1])],dims=['time','lon','lat'])
        idayX=xr.DataArray(idayL,coords=[range(len(idayL))],dims=['time'])
        ds=xr.Dataset({'u_grid':u_gridX,'v_grid':v_gridX,'q_grid':q_gridX,'phi_grid':phi_gridX,'ps_grid':ps_gridX,'t_grid':tgridX,'sst_anom':sst_anomX,\
                       'iday':idayX})
        ds.to_netcdf('dataX/stateVars_%4.4i_%2.2i.nc'%(old_year,old_month),encoding={'u_grid':{'zlib':True},'v_grid':{'zlib':True},'q_grid':{'zlib':True},'phi_grid':{'zlib':True},\
                                                                                     'ps_grid':{'zlib':True},'t_grid':{'zlib':True},'sst_anom':{'zlib':True}})
        print(old_year,old_month)
        xL=[]
        u_gridL=[]
        v_gridL=[]
        q_gridL=[]
        phi_gridL=[]
        ps_gridL=[]
        tgridL=[]
        sst_anomL=[]
        idayL=[]
        #break
    itime+=1

1990-03-09 00:00:00
1990 3
1990 4
1990 5
1990 6
1990 7
1990 8
1990 9
1990 10
1990 11


In [28]:
print(len(u_gridL))
print((model.current_date-datetime(new_year,1,1,0,0,0)).days)


0
334
